In [ ]:
import polars as pl
import gzip
from pathlib import Path
from lxml import etree

In [ ]:
pl.read_ndjson('../data/papers.jsonl').drop('abstract').filter(pl.col('arxiv_id').str.contains('1207.0580'))

In [ ]:


# --- Load and parse the gzipped TEI XML ---
xml_path = Path("../data/xml_docs/1207.0580.xml.gz")
with gzip.open(xml_path, "rb") as f:
    data = f.read()

parser = etree.XMLParser(recover=True)
root = etree.fromstring(data, parser=parser)

NS = {"tei": "http://www.tei-c.org/ns/1.0"}
tei_text = lambda el: " ".join(" ".join(el.itertext()).split()) if el is not None else ""

# --- Quick helpers ---
def get_title(root):
    xp = [
        ".//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title",
        ".//tei:fileDesc/tei:sourceDesc//tei:biblStruct/tei:analytic/tei:title[@type='main']",
        ".//tei:fileDesc/tei:sourceDesc//tei:biblStruct/tei:monogr/tei:title",
    ]
    for path in xp:
        node = root.find(path, NS)
        if node is not None and tei_text(node):
            return tei_text(node)
    return None

def get_abstract(root):
    node = root.find(".//tei:text/tei:front/tei:abstract", NS) or \
           root.find(".//tei:profileDesc/tei:abstract", NS)
    return tei_text(node) if node is not None else None

def iter_sections(root):
    """Yield (path, level, title, type, text) for each section/subsection."""
    body = root.find(".//tei:text/tei:body", NS)
    if body is None:
        return
    stack = [(body, 0, [])]
    while stack:
        node, level, path = stack.pop()
        divs = list(node.findall("./tei:div", NS))
        for i, div in enumerate(divs, 1):
            new_path = path + [i]
            title = tei_text(div.find("./tei:head", NS))
            paras = [tei_text(p) for p in div.findall("./tei:p", NS)]
            text = "\n\n".join([p for p in paras if p])
            yield {
                "path": ".".join(map(str, new_path)),
                "level": level + 1,
                "title": title,
                "type": div.get("type"),
                "text": text,
            }
            # Push children for recursion
            for child in reversed(div.findall("./tei:div", NS)):
                stack.append((child, level + 1, new_path))

# --- Extract ---
title = get_title(root)
abstract = get_abstract(root)
sections = list(iter_sections(root))

print(f"# {title or '(no title)'}\n")
if abstract:
    print("## Abstract\n", abstract, "\n")

for s in sections[:10]:  # show first few sections
    hashes = "#" * min(6, s["level"] + 1)
    heading = s["title"] or f"Section {s['path']}"
    print(f"{hashes} {s['path']} — {heading}")
    if s["type"]:
        print(f"_type: {s['type']}_")
    if s["text"]:
        print(s["text"][:400] + ("…" if len(s["text"]) > 400 else ""))
    print()

pl.DataFrame(sections).head()
